정규화 ,vaildataion 추가

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

import pandas as pd 
import os
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import to_categorical

from keras.models import Sequential,load_model

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
np.random.seed(1)

In [2]:
train_datagen = ImageDataGenerator(
    validation_split = 0.2,
    rotation_range=20,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.7,
    
    rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './images2/train',
        target_size = (300,300),
        batch_size=1000,
        #color_mode ='rgb',
        classes = ['food','interior', 'exterior'],
        shuffle =True,
        seed = 42,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        './images2/test',
        classes = ['food','interior', 'exterior'],
        target_size=(300,300),
        batch_size=200,
#         color_mode ='rgb',
        shuffle =True,
        seed = 42,
        class_mode='categorical')

Found 36000 images belonging to 3 classes.
Found 9000 images belonging to 3 classes.


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),
                 activation='relu',
                 input_shape=(300,300,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.25))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(16, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
# model.add(Dense(8, activation='selu'))
#model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))

# 3. 모델 학습과정 설정 및 요약
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습시키기
model.fit_generator(
    train_generator,
    steps_per_epoch=20,
    epochs=150,
    validation_data=test_generator,
    validation_steps=20)

model.save('model-15')

# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=10)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 6. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(test_generator, steps=10)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 16)        4624      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 19600)             0

In [ ]:
print(train_generator.class_indices)
print(test_generator.class_indices)